# Содержание:
* [Общие моменты](#general)
    1. [Установка Python](#install)
    2. [Работа с виртуальными окружениями](#virtualenv)
* [Основная часть](#main)
    1. [Динамическая типизация](#dyn)
    2. [Отсутствие статического анализа кода](#static)
    3. [Немного о рефлексии](#reflex)
    4. [Изменяемые объекты](#mut)
        * Списки и операции над ними
        * Генераторы вообще
    5. [Ещё немного интересного с изменяемыми объектами](#mut2)
    6. [Функция как объект](#func)
        * Коварность mutable-аргументов
        * Вложенные функции и каррирование
        * Декораторы
        * Лямбда-функции и приложения
    7. [NumPy](#numpy)
        * Статистика и другое
        * Производительность
        * Векторизация
        * Экономия памяти
        * Удобство
        * Практическое задание #1
        * Практическое задание #2
        * Практическое задание #3


## Общие моменты <a class="anchor" id="general"></a>

### Установка Python <a class="anchor" id="install"></a>

Установить Python для Windows [можно тут](https://www.python.org/downloads/), следуя стандартным инструкциям установки, не забыв добавить исполняемый файл Python в `PATH`.

Для UNIX-based операционных систем установку можно осуществить с помощью пакетного менеджера (будь то `apt`, `apk`, `pacman` или `brew`).

С более полным гайдом по установке можно ознакомиться [тут](https://dsc.sgu.ru/machine-learning/setup/).

### Работа с виртуальными окружениями <a class="anchor" id="virtualenv"></a>

Python, как и любой другой язык, позволяет использовать пользовательские
библиотеки с помощью собственного пакетного менеджера под названием `pip`.

Но при одновременной работе с несколькими проектами установка пакетов может
привести к конфликту установленных библиотек разных версий, поэтому на практике
применяют виртуальные окружения.

Создать виртуальное окружение в текущей директории можно с помощью команды 

`python3 -m venv .venv`. 

Здесь `venv` — имя модуля, `.venv` — директория окружения. Логично создавать окружение в корневой директории проекта.

При работе в [Jupyter Notebook](https://docs.jupyter.org/en/latest/running.html) (не [Google Colab](https://colab.google/)) необходимо также убедиться в том, что вы используете именно это окружение. В Visual Studio Code это отображается в верхней правой части экрана — там указывается папка с окружением и соответствующая версия Python.

В рамках Клуба мы рекомендуем пользоваться Google Colab в случае, если он кажется наиболее удобным, либо у вас нет собственного ноутбука, поскольку Colab — веб-сервис и доступен с любого устройства.

Здесь стоит оговориться, что для Windows нам не известно какого-то очень удобного способа создавать эти окружения — можно использовать либо командную строку, либо специфичные средства создания таких окружений в текстовом редакторе, если таковые имеются.

Можно также использовать любые надстройки над `venv`-ом — [Poetry](https://python-poetry.org/), [Conda](https://docs.conda.io/en/latest/) или, например, относительно свежий проект [Rye](https://github.com/mitsuhiko/rye).

## Основная часть <a class="anchor" id="main"></a>

Далее будут рассмотрены основные особенности языка, которые принципиально важны в практической деятельности.

### Динамическая типизация (dynamic typing) <a class="anchor" id="dyn"></a>

Хорошо известно, что Python — динамических язык. В связи
с этим одна и та же переменная может принимать значения различных типов.

Поэтому, любое из следующий выражений будет вполне справедливо

In [ ]:
x = 3
x = '3'
x = ['3']
x = [['3']]

Возможность одной и той же переменной присваивать значения различных типов является одновременно и плюсом, и минусом. 

С одной стороны, это позволяет меньше задумываться о типах значений. С другой стороны, это заставляет больше задумываться о них в последствии.

In [ ]:
def f(x: int, y: str) -> dict:
    return x + y


print(f(3, 5))

Существует возможность добавлять аннотации типов, но эти аннотации имеют смысл
лишь для программиста, но не интерпретатора.

Выражение выше при заданных аргументах будет выполнено корректно, хотя `y` имеет при себе аннотацию `str`, а сам метод в теории должен возвращать `dict`.

### Отсутствие статического анализа кода (static checking) <a class="anchor" id="static"></a>

Ещё одной важной чертой Python является отсутствие полноценной системы статического анализа кода.

Иными словами — ошибки обнаруживаются не на этапе запуска программы, а во время её исполнения.

Таким образом, программа может не завершиться успешно спустя долгое время и "обрушить" значительную часть предшествующей работы программы.

In [ ]:
df = [i for i in range(int(1e6))]

SOME_RESULT = "very_meaningful_result"
SOME_TASK = "very long python is learning how to be a python"


def job(some_arg):
    
    SOME_TASK(some_arg)
    # ...VERY VERY VERY LONG TIME PASSED
    SOME_RESULT.to_csv("output.csv")  # where result is numpy array, 
                                      # not dataframe as expected

job(df)


### Немного о рефлексии <a class="anchor" id="reflex"></a>

Механизм рефлексии можно понимать как способность программы "обращаться" к самой себе в ходе исполнения — просматривать список методов или полей
объекта, дополнять и изменять классы и так далее.

В Python много полезных применений этого механизма, но среди них есть особенно полезные.

Например, функция `dir` позволяет отобразить список существующих методов
заданного класса.

In [ ]:
dir(list)

Если отсеять "магические" методы, то зачем вообще открывать документацию в
браузере? :)

In [ ]:
def vdir(obj):
    return [x for x in dir(obj) if not x.startswith('__')]

methods = vdir(list)

for method in methods:
    help(getattr(list, method))

Функция `help` может отобразить документацию для заданного метода. Но так как
результат работы `dir` — названия методов, а не сами методы, то необходимо
предварительно получить сам метод с помощью функции `getattr`.

### Изменяемые объекты (mutable objects) <a class="anchor" id="mut"></a>

Проще всего мыслить так. Изменяемые объекты это:
1. Списки
2. Словари
3. Множества
4. Пользовательские классы

Все остальные классы можно причислить к неизменяемым.

In [ ]:
# Mutable
d = {"a": 1, "b": 2, "c": 3, "d": 4}

# Mutable
lst = d.values()

# Mutable
s = set(lst)

# Will it work?
# c = {[1, 2, 3]: "[1, 2, 3]"}

#### Списки и операции над ними

Срезы списков (slices)

In [ ]:
x = [1, 2, 3, 4, 5]

print(x[1:3])
print(x[:-1]) # or x[:len(x) - 1]
print(x[0::2])

Добавление элемента/списка

Операция `append` с точностью до названия присутствует практически в каждом
языке программирования, а вот `extend` встречается не везде. В целом, смысл
функций должен быть вполне потянен исходя из вывода интерпретатора.  

In [ ]:
a = [1, 2, 3, 4, 5]
b = [1, 2, 3]

print(a.append(b))
print(a)
print(a.extend(b))
print(a)

Генераторы списков (list comprehension)

Генераторы списков — специальная конструкция, которая позволяет компактно
создавать списки. Ключевым оператором здесь выступает `in` — который требует
генератор и возвращает... список? Или генератор? Давайте посмотрим.

In [ ]:
print([x * x for x in range(10)])

А что если опустить квадратные скобки?

In [ ]:
print(x * x for x in range(10))

Давайте сохраним этот объект

In [ ]:
gen = (x * x for x in range(10))

И преобразуем в его список

In [ ]:
list(gen)

#### Генераторы вообще

Генераторы, на самом деле, могут быть сколь угодно сложными. Если синтаксически
Python не позволяет реализовать нужную функциональность, то можно описать поведение
генератора в виде функции.

In [ ]:
def traverse(obj):
    if type(obj) != list:
        yield obj
    else:
        for subobj in obj:
            for element in traverse(subobj):
                yield element

Ключевое слово `yield` похоже на `return` — оно тоже возвращает значение, но
отличие в том
Его наличие гарантирует, что результатом вызова функции будет генератор. 

Кроме того,  `yield` приостанавливает выполнение функции до тех пор, пока
от генератора не понадобится следующее значение генератора.

Тогда `t` — генератор, и мы все ещё можем сгенерировать список на его основе:

In [ ]:
t = traverse([[1, 2, 3], 2, [[3]]])
list(t)

Можно попробовать взять четыре элемента генератора, а не все. 

Для этого можно воспользоваться методом `next`, который возвращает следующий
элемент итератора.

In [ ]:
[next(t) for _ in range(4)]

Ничем хорошим это не закончится, поскольку генератор — вещь, имеющая состояние
и оно не обнуляется между операциями. 

"Обнулить" его можно, например, пересоздав итератор.

In [ ]:
t = traverse([[1, 2, 3], 2, [[3]]])
[next(t) for _ in range(4)]

Генераторы dict-ов (dict comprehension)

То же выражение генератора можно обернуть и в привычные нам фигурные скобки, 
и мы получим словарь.

In [ ]:
{x + y: x * y for x in [1, 2, 3] for y in [5, 7]}

### Ещё немного интересного с изменяемыми объектами <a class="anchor" id="mut2"></a>

Хотим скопировать список. Так можно?

In [ ]:
x = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

y = x

y[0] = [2, 3, 4]

x

А так?

In [ ]:
y = list(x)

y[0] = [1, 2, 3]
#y[0][1] = [2, 3, 5]

x

Может, что-то такое?

In [ ]:
y = x.copy()

y[0][1] = [2, 3, 6]

x

Ну или такое...?

In [ ]:
from copy import deepcopy

y = deepcopy(x)

y[0][1] = 7

x

### Функция как объект (functions) <a class="anchor" id="func"></a>

#### Коварность mutable-аргументов по умолчанию (mutable default arguments)

Известно, что Python может принимать аргументы по умолчанию. Но у изменяемых
аргументов по умолчанию есть особенности.

In [ ]:
# Append all odd numbers less than n to x or use empty if not passed
def f(n, x=[]):
    for i in range(0, n, 2):
        x.append(i)

    return x

print(f(6))
print(f(4))
print(f(4, []))

Давайте попробуем вывести в интерпретатор, что из себя представляет функция.

In [ ]:
f

Оказывается, что функция — это объект cо своими полями и методами. 

Среди полей есть переменная `__defaults__`, в котором и сокрыта вся магия такого
поведения.

In [ ]:
f.__defaults__

Оказывается, что такое поведение обусловлено тем, что аргумент по умолчанию
представляет из себя поле класса, которое, очевидно, не обновляется с каждым
вызовом функции.

#### Вложенные функции и каррирование (nested functions)

Python также позволяет объявлять вложенные функции, что в совокупности с тем,
что функция представляет собой объект, позволяет реализовывать т.н. каррирование.

Говоря простым языком, это когда одна функция (например, двух аргументов) возвращает
другую функцию (соответственно, одного аргумента).

In [ ]:
def f(x):
    def g(y):
        return x + y
    return g
    
add_5 = f(5)
print(add_5(3))

#### Декораторы (decorators)

Декораторы — это функции, которые принимают на вход функцию и модифицирует или
дополняет её поведение. 

In [ ]:
def add_one_more(call):
    def wrapper(*args, **kwargs):
        return call(*args, **kwargs) + 1
    
    return wrapper


@add_one_more
def h(x):
    return x + 1

h(3)

Например, декоратор `add_one_more` добавляет дополнительную
единицу к результату вызова функции, к которой она применяется.

А вот пример применения декоратора в промышленном программировании

todo добавить пару комментариев в блоке

In [ ]:
def verifiable_request(call):
    async def wrapper(*args, **kwargs):
        # Получаем результат функции-аргумента декоратора.
        response = await call(*args, **kwargs)
        # Если код не соответствует нормальному ответу, то особым образом
        # обрабатываем это
        if response.status_code != status.HTTP_200_OK:
            logger.error(
                f"Got {response.status_code} after calling to {call.__name__} (args['uuid'])"
            )
            raise HTTPException(
                status.HTTP_500_INTERNAL_SERVER_ERROR,
                detail="One of a services is unavailable at the moment.",
            )
        # Иначе возвращаем тело ответа
        return response.json()

    return wrapper

In [ ]:
@verifiable_request
async def call_scraper(uuid: UUID, request: Request):
...
async with httpx.AsyncClient() as client:
        return await client.post(url, json=body, timeout=30)

Декоратор `verifiable_request` применяется к другим функциям, осуществляющим
некоторый запрос по сети. Если запрос завершается успешно, он возвращает
результат. 

Но если в результате вызова функции код ответа не 200, то декоратор бросает
исключение с необходимым текстом.

#### Лямбда-функции и приложения (lambda-functions/lambda-expressions)

Лямбда-функции — это неименованные функции. Чаще всего они применяются для
описания однострочных преобразований, либо сложных, но использующихся лишь
раз.

In [ ]:
three_x_plus_1 = lambda x: 3 * x + 1 \
    if x % 2 == 1 else x // 2

three_x_plus_1

Эта функция — тоже объект, поэтому его можно сохранить в переменную, как и
любую другую, и затем получить её значение от некоторого аргумента.

In [ ]:
three_x_plus_1(3)

Кстати, вышеописанная функция занимает большое место в математике. С ней связана т.н.
гипотеза Коллатца, которая утверждает о том, что для любого числа есть конечное число итераций из применений этой функции к её предыдущему результату снова и снова, необходимое для того, чтобы свести любое число к циклу 1-2-4.

Подробнее об этой гипотезе можно [есть очень хороший видеоролик](https://www.youtube.com/watch?v=QgzBDZwanWA&ab_channel=VertDider). 

Давайте опишем функцию `iterate_until`, которая будет итеративно получать
последующие композиции некоторой функции до тех пор, пока не будет соблюдено
некоторое условие для заданного числа.

In [ ]:
def iterate_until(f, condition):
    def iterate(x):
        iterations = 0
        while True:
            x = f(x)
            if condition(x):
                return iterations
            iterations += 1
            
    return iterate

Используя в качестве функции `three_x_plus_1` и условие `x == 1` получим
процесс, который и описывается в гипотезе Коллатца.

In [ ]:
iterate_until(three_x_plus_1, lambda x: x == 1)(989345275647)

Теперь мы можем посчитать, сколько итераций может понадобиться до сведения
каждого числа до цикла 1-2-4. Давайте в качестве примера возьмем все числа
до $10^7$.

In [ ]:
x = range(1, int(1e7))

map_func = iterate_until(three_x_plus_1, lambda x: x == 1)

iterations = map(map_func, x)
iterations

Лямбда-функции часто применяются вместе с встроенными функциями-генераторами. В
частности, функция `map`.

Применяя `map` массив чисел от $1$ до $10^7$ получаем массив, где вместо чисел —
соответствующее им число итераций.

In [ ]:
iterations = list(iterations)

### NumPy <a class="anchor" id="numpy"></a>

Библиотека NumPy позволяет выполнять общие математические операции над
массивами данных.

Библиотека использует скомпилированные на С функции. 

Давайте посмотрим, чем нам может быть интересен NumPy.

In [ ]:
import numpy as np

#### Статистика и другое

С помощью NumPy мы легко можем получить стандартные величины, такие как
среднее, максимальное, минимальное и другие статистические величины.

Так, например, мы можем получить среднее число итераций, необходимых до сходимости
до 1 чисел из задачи Колллатца.

In [ ]:
print(np.max(iterations))
print(np.mean(iterations))

#### Производительность

NumPy является высокопроизводительным модулем, и это достаточно легко
продемонстрировать наглядно.

Давайте для примера перемножим два вектора размера $10^7$.

In [ ]:
import random

a = [random.randint(1, 100) for _ in range(int(1e7))]
b = [random.randint(1, 100) for _ in range(int(1e7))]

##### Векторизация

In [ ]:
%timeit -n 10 [a_i * b_i for a_i, b_i in zip(a, b)]

In [ ]:
a_np = np.array(a, dtype=np.int16)
b_np = np.array(b)

In [ ]:
%timeit -n 10 a_np * b_np

#### Экономия памяти

In [ ]:
import sys

sys.getsizeof(a)

In [ ]:
sys.getsizeof(a_np)

#### Удобство

Скалярные, векторные и матричные операции (scalar, vector and matrix operations)

In [ ]:
x = np.array([1, 2, 3])
mat = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

print(x + 1)
print(x * 3)
print(x + x)

Манипуляции с представлением данных

In [ ]:
print(mat.reshape(3, 3))
print(mat.reshape(-1, 1))

Операции из линейной алгебры

In [ ]:
# Transposing

print(mat.T)
print(np.linalg.inv(mat))

#### Практическое задание #1 (min-max нормализация)

Дан список чисел. 

Необходимо:
1. преобразовать его к массиву NumPy
2. вычислить максимальное и минимальное значение массива
3. изменить каждое значение $x_i$ массива по формуле 

\begin{equation}
x_i = \frac{x_i - min}{max - min}
\end{equation}

для этого необходимо использовать `np.vectorize`, аргументом которой объявить
лямбда-функцию, описывающую формулу (1).

#### Практическое задание #2 (Гипотеза NumPy-Коллатца)

Ранее была описана гипотеза Коллатца. Выполните аналогичные ранее действия, используя `np.array` и операции NumPy вместо обычных.

Оцените вклад NumPy в производительность такого решения. Попытайтесь объяснить себе, почему был получен именно такой результат и поделитесь ответом с кем-то из организаторов Клуба, нам ведь интересно :)

#### Практическое задание #3 (факультатив)

Mojo — это новый язык программирования, который создан специально для ML и призван решить одну из главных проблем в Python, а именно его плохую производительность.

[Здесь](https://gist.github.com/eugeneyan/1d2ea70fed81662271f784034cc30b73) приведен сравнение производительности для нагруженных матричными и векторными операциями задач. 

Конечно, он не совсем справедлив, поскольку в среднем выигрыш будет не настолько велик, но тем не менее связанные с машинным обучением операции потенциально могут работать в сотни и тысячи раз быстрее.


В качестве факультативного задания можно ознакомиться с основами этого языка в [этом видео](https://www.youtube.com/watch?v=5Sm9IVMet9c&t=6771s&ab_channel=freeCodeCamp.org).